# NASA Requests

In [ ]:
#Downalod e importação dos arquivos de log. 
URL = 'ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz'
from urllib.request import urlretrieve
urlretrieve(URL, 'NASA_access_log_Jul95.gz')

In [ ]:
URL = 'ftp://ita.ee.lbl.gov/traces/NASA_access_log_Aug95.gz'
from urllib.request import urlretrieve
urlretrieve(URL, 'NASA_access_log_Aug95.gz')

In [1]:
import gzip
import shutil
with gzip.open('NASA_access_log_Jul95.gz', 'rb') as f_in:
    with open('NASA_access_log_Jul95.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [2]:
import gzip
import shutil
with gzip.open('NASA_access_log_Aug95.gz', 'rb') as f_in:
    with open('NASA_access_log_Aug95.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [3]:
import csv
with open('NASA_access_log_Jul95.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=' ')

In [4]:
import csv
with open('NASA_access_log_Aug95.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=' ')

In [5]:
print(readCSV)

In [6]:
import pandas as pd
import numpy as np
import csv
column_names = ['Host','1','2','Timestamp','Timezone','Requisicao','Retorno_HTTP','Total_de_bytes_retornados']
data = pd.read_csv('NASA_access_log_Jul95.csv', names=column_names, delimiter=' ', encoding='ISO-8859-1', low_memory=False)
data.head()

In [31]:
data2 = pd.read_csv('NASA_access_log_Aug95.csv', names=column_names, delimiter=' ', encoding='ISO-8859-1', low_memory=False)
data2.head()

,Host,1,2,Timestamp,Timezone,Requisicao,Retorno_HTTP,Total_de_bytes_retornados
0,in24.inetnebr.com,-,-,[01/Aug/1995:00:00:01,-0400],GET /shuttle/missions/sts-68/news/sts-68-mcc-0...,200,1839
1,uplherc.upl.com,-,-,[01/Aug/1995:00:00:07,-0400],GET / HTTP/1.0,304,0
2,uplherc.upl.com,-,-,[01/Aug/1995:00:00:08,-0400],GET /images/ksclogo-medium.gif HTTP/1.0,304,0
3,uplherc.upl.com,-,-,[01/Aug/1995:00:00:08,-0400],GET /images/MOSAIC-logosmall.gif HTTP/1.0,304,0
4,uplherc.upl.com,-,-,[01/Aug/1995:00:00:08,-0400],GET /images/USA-logosmall.gif HTTP/1.0,304,0


In [9]:
#Junção dos 2 logs num único dataset 
frames = [data, data2]
result = pd.concat(frames)
result.head()

,Host,1,2,Timestamp,Timezone,Requisicao,Retorno_HTTP,Total_de_bytes_retornados
0,199.72.81.55,-,-,[01/Jul/1995:00:00:01,-0400],GET /history/apollo/ HTTP/1.0,200,6245
1,unicomp6.unicomp.net,-,-,[01/Jul/1995:00:00:06,-0400],GET /shuttle/countdown/ HTTP/1.0,200,3985
2,199.120.110.21,-,-,[01/Jul/1995:00:00:09,-0400],GET /shuttle/missions/sts-73/mission-sts-73.ht...,200,4085
3,burger.letters.com,-,-,[01/Jul/1995:00:00:11,-0400],GET /shuttle/countdown/liftoff.html HTTP/1.0,304,0
4,199.120.110.21,-,-,[01/Jul/1995:00:00:11,-0400],GET /shuttle/missions/sts-73/sts-73-patch-smal...,200,4179


In [10]:
#Pergunta 1:Numero de hosts unicos 
df = result.groupby('Host').nunique()
df['Host'].count()

137979

In [11]:
#Pergunta 2: Total de erros 404 
result.groupby('Retorno_HTTP').Retorno_HTTP.count()

Retorno_HTTP
200                  3100521
302                    73070
304                   266773
400                       15
403                      224
404                    20872
500                       65
501                       41
HTTP/1.0"                 29
images/ssbuv1.gif          1
of                         1
Name: Retorno_HTTP, dtype: int64

In [28]:
#Pergunta 3: Os 5 URLs que mais causaram erro 404
is_404 = result['Retorno_HTTP'].str.contains('404', case=False, na=False)
result[is_404].groupby('Host').Host.count().sort_values(ascending=False).head(5)

Host
hoohoo.ncsa.uiuc.edu           251
piweba3y.prodigy.com           156
jbiagioni.npt.nuwc.navy.mil    132
piweba1y.prodigy.com           114
www-d4.proxy.aol.com            91
Name: Host, dtype: int64

In [46]:
import datetime as dt
from datetime import datetime

date_str1 = df.['Timestamp'].iloc(0)
date_dt1 = datetime.strptime(date_str1, '[%d/%b/%Y:%H:%M:%S')
date_dt1

datetime.datetime(1995, 7, 1, 0, 0, 1)

In [49]:
#Pergunta 4: Quantidade de erros 404 por dia
#result[is_404].groupby('Timestamp').Host.count()
df = result
df['date'] = df['Timestamp'].dt.date
df.head()

TypeError: strptime() argument 1 must be str, not Series

In [ ]:
#Pergunta 5: A soma total dos bytes retornados
result.Total_de_bytes_retornados.sum()